In [ ]:
import os
import numpy as np
import raster_tools as rt 
import rasterio
import matplotlib.pyplot as plt

In [1]:
input_dir = "base_raster"
target_dir = "processed_raster"

## Read Raster as Memmory Map

In [ ]:
src = rasterio.open(input_dir + "israelOrtho_20.tif")
rast = np.memmap("rast.npy",shape=(src.count ,src.height,src.width),dtype=np.float64, mode="w+")
rast[:] = src.read()

## Convert Raster to Grayscale

In [ ]:
rast_grayscale = np.memmap("rast_grayscale.npy",shape=rast.shape[1:], dtype=np.float64, mode="w+")
rt.img2grayscale(rast,(0.299, 0.587, 0.114), rast_grayscale)

## Calculate 3x3 and 9x9 Standard Deviation Filter

In [ ]:
rast_std3x3 = np.memmap("rast_std3x3.npy",shape=rast.shape[1:], dtype=np.float64, mode="w+")
rt.std_filter(rast_grayscale,3, target_rast = rast_std3x3)

In [ ]:
rast_std9x9 = np.memmap("rast_std9x9.npy",shape=rast.shape[1:], dtype=np.float64, mode="w+")
rt.std_filter(rast_grayscale,9, rast_std9x9)

In [ ]:
rast_std3x3_normalized = np.memmap("rast_std3x3_normalized.npy",
    shape=rast.shape[1:], dtype=np.float64, mode="w+")
rast_std3x3_normalized[:] = rt.normalize(rast_std3x3)

rast_std9x9_normalized = np.memmap("rast_std9x9_normalized.npy",
    shape=rast.shape[1:], dtype=np.float64, mode="w+")
rast_std9x9_normalized[:] = rt.normalize(rast_std9x9)

## Smooth the Std-Filterd Raster using 9x9 Average Filter and use Median Filter afterwords
### The information could give context for traversability as well! The higher the Deviatian the more rough is the surface => lower traversability

In [ ]:
rast_std9x9_mean = np.memmap("rast_std9x9_mean.npy",shape=rast.shape[1:], dtype=np.float64, mode="w+")
rt.avg_filter(rast_std9x9_normalized, 9, target_rast =rast_std9x9_mean)

In [ ]:
rast_std9x9_mean_med = np.memmap("rast_std9x9_mean_med.npy",
    shape=rast.shape[1:], dtype=np.float64, mode="w+")
rt.med_filter(rast_std9x9_mean,9, rast_std9x9_mean_med)

## Normalize the STD-Raster between 0 & 1

In [ ]:
rast_std9x9_mean_med_normalized = np.memmap("rast_std9x9_mean_med_normalized.npy",
    shape=rast.shape[1:], dtype=np.float64, mode="w+")

rast_std9x9_mean_med_normalized[:] = rt.normalize(rast_std9x9_mean_med)

## Export Rasters as Geotiff

In [ ]:
profile = src.profile
profile.update(dtype=np.float64,count=1)

with rasterio.open(target_dir + "rast_std3x3.tif","w+",**profile) as dst:
    dst.write(rast_std3x3_normalized.astype(np.float64),1)

with rasterio.open(target_dir + "rast_std9x9.tif","w+",**profile) as dst:
    dst.write(rast_std9x9_normalized.astype(np.float64),1)

with rasterio.open(target_dir + "stdMeans_medfilt_normalized.tif","w+",**profile) as dst:
    dst.write(rast_std9x9_mean_med_normalized.astype(np.float64),1)

## Clean Up Temporary-Files

In [ ]:
for f in os.listdir():
    if f.endswith(".npy"):
        os.remove(f)

    if f.startswith(".fuse"):
        os.remove(f)

In [ ]:
src = rasterio.open(target_dir + "rast_std3x3.tif")
rast = src.read(1)

In [ ]:
rast_std = rast.std()

In [ ]:
tresh = rast_std * 3

In [ ]:
profile = src.profile
profile.update(count=1,dtype=np.uint8)

with rasterio.open(target_dir + "rast_std_bin3x3.tif","w+",**profile) as dst:
    dst.write((rast > tresh).astype(np.uint8), 1)